<div class="alert alert-block alert-info">
    <h1>Using the AρρEEARS API in an ECOSTRESS ARD Workflow - Getting Started</h1>
</div>

---
## Objective
The intent of this tutorial is to familiarize Landsat Analysis Ready Data ([ARD](https://www.usgs.gov/land-resources/nli/landsat/us-landsat-analysis-ready-data?qt-science_support_page_related_con=0#qt-science_support_page_related_con)) users with the [AρρEEARS](https://lpdaac.usgs.gov/tools/appeears/) application programming interface (API) with demonstrations on how the API, and the services it provides, can be leveraged in an analysis workflow.

---
## Use Case
This tutorial was developed using a real-world use case for a project being completed by the [NASA DEVELOP Node at the Marshall Space Flight Center](https://develop.larc.nasa.gov/nodes/MSFC.html). **NASA Develop is a program aimed at integrating NASA Earth observations with society to foster future innovation and cultivate the professionals of tomorrow by addressing diverse environmental issues today.** 

The example use case comes from a project titled, "Utilizing NASA Earth Observations to Assess Coastline Replenishment Initiatives and Shoreline Risk along Delaware's Coasts". The group is partnering with the Delaware Department of Natural Resources and Environmental Control, Division of Watershed Stewardship for this project. The goals for the project include to identify areas of current and potential shoreline loss along the coast of Delaware, assess the current restoration efforts, and **create time-series coastline maps**. 

### Example: Submit an AppEEARS area request for a portion of the Delaware coast along the Prime Hook National Wildlife Refuge to extract Landsat Analysis Ready Data for the years before and after Hurricane Sandy. The outputs will be used to generate false color composite time series to visualize changes to the coastline during the time period. 

---
## Topics Covered
1. [**Getting Started**](#gettingstarted)  
    1.1 [Enable Access to the API](#1.1)  
    1.2 [Login](#1.2)  
2. [**Submit an Area Request**](#submittask)  
    2.1 [Import a Shapefile](#2.1)  
    2.2 [Compile the JSON payload to submit to AρρEEARS](#2.2)  
    2.3 [Submit a task request](#2.3)  
    2.4 [Get task status](#2.4)  
3. [**Download a Request [Bundle API]**](#downloadrequest)  
    3.1 [List files associated with the request](#3.1)  
    3.2 [Download files in a request](#3.2)  
4. [**Explore AρρEEARS Outputs**](#explore)  
    4.1 [Open and explore data using xarray](#4.1)  
    4.2 [Visualize Time Series Data](#4.2)  
5. [**Quality Filtering**](#qualityfiltering)  
    5.1 [Decode quality values](#5.1)  
    5.2 [Create and apply quality mask](#5.2)  
    5.3 [Plot quality filtered data](#5.3)  

## AρρEEARS Information
To access AρρEEARS, visit: https://lpdaacsvc.cr.usgs.gov/appeears/

For comprehensive documentation of the full functionality of the [AρρEEARS API](https://lpdaacsvc.cr.usgs.gov/appeears/api/), please see the AρρEEARS API Documentation: https://lpdaacsvc.cr.usgs.gov/appeears/api/

Throughout the exercise, specific sections of the API documentation can be accessed by clicking the hyperlinked text.

## Setup and Dependencies 
It is recommended to use [Conda](https://conda.io/docs/), an environment manager to set up a compatible Python environment. Download Conda for your OS here: https://www.anaconda.com/download/. Once you have Conda installed, Follow the instructions below to successfully setup a Python environment on MacOS or Windows.

This Python Jupyter Notebook tutorial has been tested using Python versions 3.6, 3.6.6 and 3.7.

Conda was used to create the python environment.  
- **Option 1**: Download the [environment yml file](https://git.earthdata.nasa.gov/projects/LPDUR/repos/landsat-ard-appeears-api/browse/environment.yml):  
    - Open the environment.yml file and change the prefix (last line) to the directory on your OS where you want to create the environment (ex: C:\Username\Anaconda3\envs\ardtutorial) and save the environment file.  
    - Using Command Prompt, Anaconda Prompt, Cmder, Terminal, or your preferred command line interface, navigate to the directory where you saved the `environment.yml` file.  
    - Type `conda env create -f environment.yml`  
    - Type `activate ardtutorial`


- **Option 2**: Download each package separately
    - Windows OS  or macOS
    `conda create -n ardtutorial python=3.6`  
- If you already had conda installed on your OS, it is recommended that you update to the latest version:  
    `conda update -n base -c defaults conda`
- Required Python packages were installed from the conda-forge channel. Installing packages from the conda-forge channel is done by adding conda-forge to your channels with:  
`conda config --add channels conda-forge`  
- Activate the newly created environment using the command: `activate ardtutorial`  
- Required packages needed for this exercise are listed below. 
    - requests  
    `conda install requests`  
    - pandas  
    `conda install pandas`  
    - geopandas  
    `conda install geopandas`  
    - xarray  
    `conda install xarray`  
    - numpy  
    `conda install numpy`  
    - netcdf4  
    `conda install netcdf4`  
    - holoviews  
    `conda install holoviews`  
    - pyviz &emsp;&emsp;**NOTE** - [PyViz](http://pyviz.org/) is installed using the pyviz channel not conda-forge.  
    `conda install -c pyviz hvplot`  
    > If you encounter an issue downloading hvplot using conda, try `pip install pyviz hvplot`  
    
### Next, download the [Jupyter Notebook](https://git.earthdata.nasa.gov/projects/LPDUR/repos/landsat-ard-appeears-api/browse/ARD_AppEEARS_API.ipynb) and [example shapefile](https://git.earthdata.nasa.gov/projects/LPDUR/repos/landsat-ard-appeears-api/browse/PrimeHookNWR_6kmBuffer.shp) to get started.

---
# 1. Getting Started <a id="gettingstarted"></a>
If this is your first time using the [AρρEEARS API](https://lpdaacsvc.cr.usgs.gov/appeears/api/), you must first enable API access by following the instructions provided below after signing in with your [NASA Earthdata Login](https://urs.earthdata.nasa.gov/).

***
## 1.1 Enable Access to the API <a id="1.1"></a>
> To enable access to the [AρρEEARS API](https://lpdaacsvc.cr.usgs.gov/appeears/api/), navigate to the [AρρEEARS website](https://lpdaacsvc.cr.usgs.gov/appeears/). Click the *Sign In* button in the top right portion of the AρρEEARS landing page screen.  

<table><tr><td>
    <img src="https://lpdaacsvc.cr.usgs.gov/assets/images/help/image001.7f0d8820.png" />
</td></tr></table>  

> Once you are signed in, click the *Manage User* icon in the top right portion of the AρρEEARS landing page screen and select *Settings*.   

<table><tr><td>
    <img src="https://lpdaacsvc.cr.usgs.gov/assets/images/help/api/image001.3bb7c98a.png" />
</td></tr></table>  

> Select the *Enable API* box to gain access to the AρρEEARS API.  

<table><tr><td>
    <img src="https://lpdaacsvc.cr.usgs.gov/assets/images/help/api/image002.ebbb9431.png" />
</td></tr></table>

## 1.2 Login to AρρEEARS/Earthdata <a id="1.2"></a>
> To submit a request, you must first [login](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#login) to the AρρEEARS API using your Earthdata login credentials.  We’ll use the `getpass` package to conceal our Earthdata login username and password. When executed, the code below will prompt you to enter your username followed by your password and store them as variables.

#### Import the required packages and set the input/working directory to run this Jupyter Notebook locally.

In [ ]:
# Import required Python packages
import requests
import getpass
import time
import os
import cgi
import json
import pandas as pd
import geopandas as gpd
import xarray
import numpy as np
import hvplot.xarray
import holoviews as hv

In [ ]:
# Set input directory, change working directory
inDir = "/scratch/rave/"  # Set input directory to the current working directory
os.chdir(inDir)               # Change to working directory

<div class="alert alert-block alert-warning" >
<b>If you have downloaded the tutorial materials to a different directory than the Jupyter Notebook, `inDir` above needs to be changed.</b>
</div> 

#### To submit a request, you must first login to the AρρEEARS API. Use the `getpass` package to enter your NASA Earthdata login **Username** and **Password**. When prompted after executing the code block below, enter your username followed by your password.

In [ ]:
# Enter Earthdata login credentials
username = getpass.getpass('Earthdata Username:')
password = getpass.getpass('Earthdata Password:')

In [ ]:
API = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'  # Set the AρρEEARS API to a variable

#### Use the `requests` package to post your username and password. A successful login will provide you with a token to be used later in this tutorial to submit a request. For more information or if you are experiencing difficulties, please see the [API Documentation](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#login).

In [ ]:
# Insert API URL, call login service, provide credentials & return json
login_response = requests.post(f"{API}/login", auth=(username, password)).json() 
del username, password
login_response

#### Above, you should see a Bearer Token. The Bearer Token is needed to leverage the AρρEEARS API via HTTP request methods (e.g., POST and GET). Notice that this token will expire approximately 48 hours after being acquired. 

In [ ]:
# Assign the token to a variable
token = login_response['token']
head = {'Authorization': f"Bearer {token}"} 
head

# 2. Submit an Area Request <a id="submittask"></a>
The [Tasks](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#tasks) service, among other things (see below), is used to submit requests (e.g., POST and GET) to the AρρEEARS system. Each call to the [Tasks](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#tasks) service is associated with your user account. Therefore, each of the calls to this service require an authentication token. The [*submit task*](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#submit-task) API call provides a way to submit a new request. It accepts data via JSON, query string, or a combination of both. In the example below, we will compile a json and submit a request.

No download commented out since it takes a while

In [ ]:
# from datapackage import Package

# package = Package('https://datahub.io/core/geo-countries/datapackage.json')

# countries = package.get_resource("countries")

# This takes about 2 minutes, downloads countries geojson in memory.

# all_countries_bytes = countries.raw_read()

# all_countries_json = json.loads(all_countries_bytes)

# Below we can iterate to get the lis tof countries in our data package.

# for i in all_countries_json['features']:
#     print(i['properties']['ADMIN'])

# for i in all_countries_json['features']:
#     if i['properties']['ADMIN'] == "France":
#         france_geo = i

# france_geo

Download using data-cli, instructions at: https://datahub.io/core/geo-countries

In [ ]:
with open(f"{inDir}core/geo-countries/archive/countries.geojson", "rb") as f:
    all_countries_geojson = json.loads(f.read())


In [ ]:
for i in all_countries_geojson['features']:
    if i['properties']['ADMIN'] == "France":
        france_geo = i


In [ ]:
france_geo

## 2.2 Compile the JSON payload to submit to AρρEEARS <a id="2.2"></a>
> Many of the required items needed in the AρρEEARS API request payload have multiple options. For example, AρρEEARS has several projections that can be selected for the output. We can use the AρρEEARS API to find out what projections are available. In this example, we are explicitly assigning our projection to the **proj** variable. To find out how to use the AρρEEARS API to list the available options for each parameter, check out the [AρρEEARS API Tutorials](https://git.earthdata.nasa.gov/projects/LPDUR/repos/appeears-api-getting-started/browse) produced by the [LP DAAC](https://lpdaac.usgs.gov/).

Listing products

In [ ]:
import requests

response = requests.get('https://lpdaacsvc.cr.usgs.gov/appeears/api/product')
product_response = response.json()
# create a dictionary indexed by the product name and version
products = {p['ProductAndVersion']: p for p in product_response}


In [ ]:
products.keys()

In [ ]:
products['CU_LC08.001']

In [ ]:
products['ECO3ANCQA.001']

In [ ]:
products['ECO3ETPTJPL.001']

In [ ]:
products['ECO4ESIPTJPL.001']

In [ ]:
products['ECO4WUE.001']

In [ ]:
import requests

product_id = 'ECO4WUE.001'

def get_layer_from_productid(product_id):
    """only works for single layer products"""
    response = requests.get('https://lpdaacsvc.cr.usgs.gov/appeears/api/product/{0}'.format(product_id))
    layer_response = response.json()
    print(layer_response)
    return layer_response
get_layer_from_productid(product_id)

In [ ]:
get_layer_from_productid('ECO4ESIPTJPL.001')

In [ ]:
get_layer_from_productid('ECO3ETPTJPL.001').keys()

In [ ]:
get_layer_from_productid('ECO3ANCQA.001').keys()

In [ ]:
task_name = 'France ET'    # User-defined name of the task
task_type = 'area'                                    # Type of task, area or point
proj = "geographic"                             # Set output projection 
outFormat = 'geotiff'                                 # Set output file format type
startDate = '08-31-2019'                              # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '10-30-2019'                                # End of the date range for which to extract data: MM-DD-YYYY
recurring = False                                     # Specify True for a recurring date range

# Define the products and layers desired
prodLayer = [{
        "layer": "L3_L4_QA_ECOSTRESS_L2_QC",
        "product": "ECO3ANCQA.001"
      },
      {
        "layer": "EVAPOTRANSPIRATION_PT_JPL_ETdaily",
        "product": "ECO3ETPTJPL.001"
      },
      {
        "layer": "EVAPOTRANSPIRATION_PT_JPL_ETinstUncertainty",
        "product": "ECO3ETPTJPL.001"
      },
      {
        "layer": "Evaporative_Stress_Index_PT_JPL_PET",
        "product": "ECO4ESIPTJPL.001"  
      }]

if tasks are too big for appeears then order one product at a time and/or restrict by date

In [ ]:
prodLayer = [{
        "layer": "L3_L4_QA_ECOSTRESS_L2_QC",
        "product": "ECO3ANCQA.001"
      }]

In [ ]:
prodLayer = [
      {
        "layer": "EVAPOTRANSPIRATION_PT_JPL_ETdaily",
        "product": "ECO3ETPTJPL.001"
      }]

In [ ]:
prodLayer = [
      {
        "layer": "Evaporative_Stress_Index_PT_JPL_PET",
        "product": "ECO4ESIPTJPL.001"  
      }]

#### Compile the JSON to be submitted as an area request.   
Notice that `primehookNWR` is inserted from the shapefile transformed to a geojson via the `geopandas` and `json` packages above in section 2.1. 

In [ ]:
france_geo

In [ ]:
%matplotlib inline
france_gdf = gpd.GeoDataFrame.from_features([france_geo]).explode()

In [ ]:
france_gdf[france_gdf.area==max(france_gdf.area)].plot()

In [ ]:
aoi = france_gdf[france_gdf.area==max(france_gdf.area)]

In [ ]:
aoi_geojson = json.loads(aoi.to_json())

In [ ]:
task = {
    'task_type': task_type,
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat}, 
                         'projection': proj},
         'geo': aoi_geojson,
    }
}

> The **task** object is what we will submit to the AρρEEARS system.

## 2.3 Submit a task request <a id="2.3"></a> 
> We will now submit our **task** object to AρρEEARS using the [*submit task*](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#submit-task) API call.

In [ ]:
# Post json to the API task service, return response as json
task_response = requests.post(f"{API}/task", json=task, headers=head)
task_response.json()   

> A task ID is generated for each request and is returned in the response. Task IDs are unique for each request and are used to check request status, explore request details, and list files generated for the request.

In [ ]:
task_id = task_response.json()['task_id']
task_id

## 2.4 Get task status <a id="2.4"></a>
> We can use the [Status](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#status) service to retrieve information on the status of all task requests that are currently being processed for your account. We will use the [*task status*](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#task-status) API call with our **task_id** to get information on the request we just submitted. 

In [ ]:
status_response = requests.get(f"{API}/status/{task_id}", headers=head)
status_response.json()

> For longer running requests we can gently ping the API to get the status of our submitted request using the snippet below. Once the request is complete, we can move on to downloading our request contents.

In [ ]:
# Use while statement to ping the API every 20 seconds until a response of 'done' is returned
starttime = time.time()
while requests.get(f"{API}/task/{task_id}", headers=head).json()['status'] != 'done':
    print(requests.get(f"{API}/task/{task_id}", headers=head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(requests.get(f"{API}/task/{task_id}", headers=head).json()['status'])

# Getting Task ID at the start of a session, in case you don't do steps 1-2 above becaus ethey are already complete.

In [ ]:
task_response[3]

In [ ]:
response = requests.get(
    'https://lpdaacsvc.cr.usgs.gov/appeears/api/task', 
    headers=head)
task_response = response.json()
print(task_response)

task_id = task_response[3]['task_id']

# 3. Download a Request <a id="downloadrequest"></a>
The [Bundle](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#bundle) service provides information about completed tasks (i.e., tasks that have a status of **done**). A bundle will be generated containing all of the files that were created as part of the task request.

## 3.1 List files associated with the request  <a id="3.1"></a>
> The [list files](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#list-files) API call lists all of the files contained in the bundle which are available for download.

In [ ]:
bundle = requests.get(f"{API}/bundle/{task_id}").json()    # Call API and return bundle contents for the task_id as json
bundle

In [ ]:
bundle['task_id']

In [ ]:
def get_bundle_size_gb(bundle):
    filesizes_gb = [i['file_size']/1e9 for i in bundle['files']]
    return np.sum(filesizes_gb)

## 3.2 Download files in a request <a id="3.2"></a>
>The [download file](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#download-file) API call gives us the information needed to download all, or a subset of the files available for a request. Just as the task has a **task_id** to identify it, each file in the bundle will also have a unique **file_id** which should be used for any operation on that specific file. The `Content-Type` and `Content-Disposition` headers will be returned when accessing each file to give more details about the format of the file and the filename to be used when saving the file.

> The `bundle` variable we created has more information than we need to download the files. We will first create a python dictionary to hold the **file_id** and associated **file_name** for each file.

In [ ]:
files = {}
for f in bundle['files']: 
    files[f['file_id']] = f['file_name']    # Fill dictionary with file_id as keys and file_name as values
files

> Now we will download the files using the **file_ids** from the dictionary into an output directory.

In [ ]:
# Set up output directory on local machine
outDir = f'{inDir}ecostress-rhone/'
if not os.path.exists(outDir):
    os.makedirs(outDir)

#### Use the `files` dictionary and a `for` loop to automate downloading all of the output files into the output directory. 

In [ ]:
for file in files:
    download_response = requests.get(f"{API}/bundle/{task_id}/{file}", stream=True)                                   # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(download_response.headers['Content-Disposition'])[1]['filename'])    # Parse the name from Content-Disposition header 
    filepath = os.path.join(outDir, filename)                                                                         # Create output file path
    with open(filepath, 'wb') as file:                                                                                # Write file to dest dir
        for data in download_response.iter_content(chunk_size=8192): 
            file.write(data)
print("Downloading complete!")

# 4. Explore AρρEEARS Outputs <a id="explore"></a>
Now that we have downloaded all the files from our request, let's start to check out our data! In our AρρEEARS request, we set the output format to 'netcdf4'. As a result, we have only one output data file. We will open the dataset as an `xarray Dataset` and start to explore.

## 4.1 Open and explore data using [`xarray`](http://xarray.pydata.org/en/stable/) <a id="4.1"></a>

> [`Xarray`](http://xarray.pydata.org/en/stable/) extends and combines much of the core functionality from both the Pandas library and Numpy, hence making it very good at handling multi-dimensional (N-dimensional) datasets that contain labels (e.g., variable names or dimension names). Let's open the netcdf file with our data as an xarray object.

In [ ]:
os.listdir(outDir)

In [ ]:
ds = xarray.open_rasterio(f'{outDir}ECO3ETPTJPL.001_EVAPOTRANSPIRATION_PT_JPL_ETcanopy_doy2018225152941_aid0001.tif')

ds

In [ ]:
ds = xarray.open_dataset(f'{outDir}CU_LE07.001_30m_aid0001.nc')  # Open the L7 ARD output NC4 file from AppEEARS
ds

> Xarray has two fundamental  data structures. A `Dataset` holds multiple variables that potentially share the same coordinates and global metadata for the file (see above). A `DataArray` contains a single multi-dimensional variable and its coordinates, attributes, and metadata. Data values can be pulled out of the DataArray as a `numpy.ndarray` using the `values` attribute.

In [ ]:
type(ds)

In [ ]:
type(ds.SRB3)

In [ ]:
type(ds.SRB3.values)

> We can also pull out information for each coordinate item (e.g., lat, lon, time). Here we pull out the *time* coordinate.

In [ ]:
ds['time']

> The `cftime.DatetimeJulian` format of the time coordinate is a little problematic for some plotting libraries and analysis routines. We are going to [convert the time coordinate](https://stackoverflow.com/questions/55786995/converting-cftime-datetimejulian-to-datetime) to the more useable datetime format `datetime64`.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
datatimeindex = ds.indexes['time'].to_datetimeindex(); # Convert to datetime64
ds['time'] = datatimeindex                             # Set converted index to dataset time coordinate

## 4.2 Visualize Time Series Data <a id="4.2"></a>
#### Below, use the [`hvPlot`](https://hvplot.pyviz.org/index.html) and [`holoviews`](https://www.holoviews.org/) packages to create an interactive time series plot of the Landsat 7 ARD data.

In [ ]:
hv_ds = hv.Dataset(ds) # Convert to holoviews dataset

#### Plot the holoviews dataset as a four dimensional RGB false color composite, defining the x, y, and time dims from the coordinates, and also the fourth dimension which defines where to put each data variable in the RGB composite. 
> Here we are using a Landsat 7 false color composite combination of:  
- R = B4 (NIR)  
- G = B5 (SWIR 1)
- B = B3 (RED)  
#### This false color combination was chosen because it highlights land-water boundaries and is useful in analysis of soil conditions. Since we are interested in analyzing the coastal shoreline, this is a good combination in order to maximize our ability to delineate between land and water features and highlight the shoreline. Vegetation will appear in shades of green/orange/brown, with increasingly saturated soils appearing in very dark colors. Water will appear very dark, almost black. 

In [ ]:
# Use the .to() method to plot the holoviews dataset as an RGB, defining the x/y/z dimensions and data variables used
timeSeries = hv_ds.to(hv.RGB, kdims=["xdim", "ydim"], dynamic=True, vdims=["SRB4","SRB5","SRB3"])
timeSeries.opts(width=600, height=600)

> Above, visualize the  multidimensional (t,x,y) plot of our gridded data. Move the slide on the right to visualize the different time slices.

#### Notice in the visualization above that there are many time slices with only fill values. This is easily explained when considering that the Landsat 7 data has been tiled and gridded into an Analysis Ready Data stack. Thus, for the observations with all fill values, there are Landsat 7 data that exist in the ARD tile, however outside of the extent of our region of interest (ROI). Below, remove all observations that only contain fill values. 

In [ ]:
# Set up a list to store time slices that contain non-fill value data by going through each time slice for a variable
goodTimes = []
for i in range(len(ds.time)):
    if np.nanmean(ds.SRB4[i,:,:]) > 0:                 # Only keep time slices where mean reflectance is greater than 0
        goodTimes.append(ds.SRB4[i,:,:].time.values)   # Append time value for valid observation to list

In [ ]:
goodTimes

> Use xarray's powerful indexing method to pull out the `time` coordinates in the `goodTimes` list.

In [ ]:
ds = ds.sel(time=goodTimes)

#### Plot the holoviews dataset again without the empty time slices.

In [ ]:
hv_ds = hv.Dataset(ds)
timeSeries = hv_ds.to(hv.RGB, kdims=["xdim", "ydim"], dynamic=True, vdims=["SRB4","SRB5","SRB3"])
timeSeries.opts(width=600, height=600)

#### How about cloudy observations? Below is an example of how to filter out cloudy or poor quality pixels from the image time series. 

# 5. Quality Filtering <a id="qualityfiltering"></a>
When available, AρρEEARS extracts and returns quality assurance (QA) data for each data file returned regardless of whether the user requests it. This is done to ensure that the user possesses the information needed to determine the usability and usefulness of the data they get from AρρEEARS. The [Quality](https://lpdaacsvc.cr.usgs.gov/appeears/api/#quality) service from the AρρEEARS API can be leveraged to create masks that filter out undesirable data values. 

In [ ]:
ds

> Notice that the xarray Dataset contains a data array/variable called `PIXELQA`, which has the same dimensions/coordinates as the `SRB#` data arrays/variables. We can use the quality array to create a mask of poor-quality data. We'll use the [Quality](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#quality) service to decode the quality assurance information. 

> We'll use the following criteria to mask out poor quality data:
- Cloud (Cloud) == No
- Cloud Shadow (CS) == No

## 5.1 Decode quality values <a id="5.1"></a>
> We do not want to decode the same value multiple times. Let's extract all of the unique data values from the `PixelQA` xarray DataArray.

In [ ]:
# Quality Filtering
quality_values = pd.DataFrame(np.unique(ds.PIXELQA.values), columns=['value']).dropna()
quality_values

> The following function decodes the data values from the `PIXELQA` xarray DataArray using the [Quality](https://lpdaacsvc.cr.usgs.gov/appeears/api/#quality) service.

In [ ]:
def qualityDecode(qualityservice_url, product, qualitylayer, value):
    req = requests.get(f"{qualityservice_url}/{product}/{qualitylayer}/{value}")
    return(req.json())

> Now we will create an empty dataframe to store the decoded quality information for the masking criteria we identified above.

In [ ]:
quality_desc = pd.DataFrame(columns=['value', 'Cloud_bits', 'Cloud_description', 'CS_bits', 'CS_description'])

> The for loop below goes through all of the unique quality data values, decodes them using the quality service, and appends the quality descriptions to our empty dataframe.

In [ ]:
for index, row in quality_values.iterrows():
    decode_int = qualityDecode(f'{API}/quality',
                               "CU_LE07.001",
                               'PIXELQA',
                               str(int(row['value'])))
    quality_info = decode_int
    df = pd.DataFrame({'value': int(row['value']),
                       'Cloud_bits': quality_info['Cloud']['bits'], 
                       'Cloud_description': quality_info['Cloud']['description'], 
                       'CS_bits': quality_info['Cloud Shadow']['bits'],
                       'CS_description': quality_info['Cloud Shadow']['description']}, index=[index])

    quality_desc = quality_desc.append(df)

In [ ]:
quality_desc

## 5.2 Create and apply quality mask <a id="5.2"></a>
> Now we have a dataframe with all of the quality information we need to create a quality mask. Next, we'll identify the quality categories that we would like to keep.

In [ ]:
# Only keep observations where cloud AND cloud shadow both = no (meaning, there are no clouds/shadows present)
mask_values = quality_desc[((quality_desc['Cloud_description'] == 'No') &
                           (quality_desc['CS_description'] == 'No'))]
mask_values

> Let's apply the mask to our xarray dataset, keeping only the values that we have deemed acceptable.

In [ ]:
dsMasked = ds.where(ds['PIXELQA'].isin(mask_values['value']))
dsMasked

#### Filter out any additional observations that may be returning only fill values after applying the cloud mask.

In [ ]:
goodTimes = []
for i in range(len(dsMasked.time)):
    if np.nanmean(dsMasked.SRB4[i,:,:]) > 0:
        goodTimes.append(dsMasked.SRB4[i,:,:].time.values)
dsFinal = dsMasked.sel(time=goodTimes)

## 5.3 Plot quality filtered data <a id="5.3"></a>
> Using the same plotting functionality from above, let's see how our data looks when we mask out the undesirable pixels.

In [ ]:
hv_ds = hv.Dataset(dsFinal)
timeSeries = hv_ds.to(hv.RGB, kdims=["xdim", "ydim"], dynamic=True, vdims=["SRB4","SRB5","SRB3"])
timeSeries.opts(width=600, height=600)

### This tutorial provides a template to use for your own research workflows. Leveraging the AρρEEARS API for extracting and formatting analysis ready data and importing it directly into Python means that you can keep your entire research workflow in a single software program, from start to finish.

<div class="alert alert-block alert-info">
    <h1> Contact Information </h1>
    <h3> Material written by Cole Krehbiel$^{1}$ & Aaron Friesz$^{2}$ </h3>
    <ul>
        <b>Contact:</b> LPDAAC@usgs.gov <br> 
        <b>Voice:</b> +1-605-594-6116 <br>
        <b>Organization:</b> Land Processes Distributed Active Archive Center (LP DAAC) <br>
        <b>Website:</b> https://lpdaac.usgs.gov/ <br>
        <b>Date last modified:</b> 10-16-2019 <br>
    </ul>

$^{1}$Innovate! Inc., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G15PD00467 for LP DAAC$^{3}$.  

$^{2}$KBR Inc., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G15PD00467 for LP DAAC$^{3}$.

$^{3}$LP DAAC Work performed under NASA contract NNG14HH33I.
</div>